In [2]:
import tensorflow

ImportError: cannot import name 'OrderedDict' from 'typing' (c:\Users\amank\PycharmProjects\pythonProject2\D_L\Real_life_project\image_classification\Img_Class\lib\typing.py)

In [10]:
import shutil
import random

def split_train_test(saved_images, train_dir, test_dir, test_ratio=0.2):
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    for class_folder in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_folder)
        if os.path.isdir(class_path):
            # Create subfolders for training and testing images
            train_class_dir = os.path.join(train_dir, class_folder)
            test_class_dir = os.path.join(test_dir, class_folder)
            os.makedirs(train_class_dir, exist_ok=True)
            os.makedirs(test_class_dir, exist_ok=True)

            # Get list of images in the class folder
            images = os.listdir(class_path)
            # Shuffle the images
            random.shuffle(images)

            # Split the images into training and testing sets
            num_test_images = int(test_ratio * len(images))
            test_images = images[:num_test_images]
            train_images = images[num_test_images:]

            # Move training images to the training directory
            for image in train_images:
                src = os.path.join(class_path, image)
                dest = os.path.join(train_class_dir, image)
                shutil.move(src, dest)

            # Move testing images to the testing directory
            for image in test_images:
                src = os.path.join(class_path, image)
                dest = os.path.join(test_class_dir, image)
                shutil.move(src, dest)

            # Remove the original class folder from saved_images directory
            shutil.rmtree(class_path)

# Define directories
saved_images = r'C:\Users\atulk\PycharmProjects\pythonProject2\D_L\Real_life_project\image_classification\research\saved_images'
train_dir = r'C:\Users\atulk\PycharmProjects\pythonProject2\D_L\Real_life_project\image_classification\research\train'
test_dir = r'C:\Users\atulk\PycharmProjects\pythonProject2\D_L\Real_life_project\image_classification\research\test'

# Split and move images
split_train_test(saved_images, train_dir, test_dir)


In [ ]:
from flask import Flask, render_template, request
import base64
import os
import shutil
import threading
import uuid
import time
import random

class ImageHandler:
    def __init__(self, saved_images,train_dir, test_dir):
        self.saved_images = saved_images
        self.train_dir=train_dir
        self.test_dir=test_dir
        self.captureCount = 0

    def remove_folder(self, folder_path):
        # Function to remove the folder after 10 minutes
        def remove():
            print(f"Waiting for 10 minutes before removing the folder '{folder_path}'...")
            # Wait for 10 minutes
            time.sleep(600)
            # Check if the folder exists before trying to remove it
            if os.path.exists(folder_path):
                shutil.rmtree(folder_path)
                print(f"Folder '{folder_path}' removed successfully.")
            else:
                print(f"The folder '{folder_path}' does not exist.")
        return remove

    def process_request(self, image_data, folder_name, remove_folder_name):
        _, encoded_data = image_data.split(',', 1)
        decoded_image = base64.b64decode(encoded_data)

        # Create the directory if it doesn't exist
        os.makedirs(os.path.join(self.saved_images, folder_name), exist_ok=True)
        image_filename = str(uuid.uuid4()) + '.png'
        image_path = os.path.join(self.saved_images, folder_name, image_filename)

        with open(image_path, 'wb') as f:
            f.write(decoded_image)

        self.captureCount += 1

        if remove_folder_name:
            folder_path = os.path.join(self.saved_images, remove_folder_name)
            if os.path.exists(folder_path):
                # Start a thread to remove the folder after 10 minutes
                remove_thread = threading.Thread(target=self.remove_folder(folder_path))
                remove_thread.start()
            else:
                print(f"The folder '{remove_folder_name}' does not exist.")

        # Get all the image URLs in the folder
        images = [os.path.join(self.saved_images, folder_name, image) for image in os.listdir(os.path.join(self.saved_images, folder_name))]

        # Pass the list of image URLs to the template
        return images

def split_train_test( self, test_ratio=0.2):
    os.makedirs(self.train_dir, exist_ok=True)
    os.makedirs(self.test_dir, exist_ok=True)

    for class_folder in os.listdir(self.saved_images):
        class_path = os.path.join(self.saved_images, class_folder)
        if os.path.isdir(class_path):
            # Create subfolders for training and testing images
            train_class_dir = os.path.join(train_dir, class_folder)
            test_class_dir = os.path.join(test_dir, class_folder)
            os.makedirs(train_class_dir, exist_ok=True)
            os.makedirs(test_class_dir, exist_ok=True)

            # Get list of images in the class folder
            images = os.listdir(class_path)
            # Shuffle the images
            random.shuffle(images)

            # Split the images into training and testing sets
            num_test_images = int(test_ratio * len(images))
            test_images = images[:num_test_images]
            train_images = images[num_test_images:]

            # Move training images to the training directory
            for image in train_images:
                src = os.path.join(class_path, image)
                dest = os.path.join(train_class_dir, image)
                shutil.move(src, dest)

            # Move testing images to the testing directory
            for image in test_images:
                src = os.path.join(class_path, image)
                dest = os.path.join(test_class_dir, image)
                shutil.move(src, dest)

            # Remove the original class folder from saved_images directory
            shutil.rmtree(class_path)

# Define directories
saved_images = r'C:\Users\atulk\PycharmProjects\pythonProject2\D_L\Real_life_project\image_classification\research\saved_images'
train_dir = r'C:\Users\atulk\PycharmProjects\pythonProject2\D_L\Real_life_project\image_classification\research\train'
test_dir = r'C:\Users\atulk\PycharmProjects\pythonProject2\D_L\Real_life_project\image_classification\research\test'

# Initialize ImageHandler
image_handler = ImageHandler(saved_images)

# Split and move images
split_train_test(saved_images, train_dir, test_dir)


In [ ]:
from flask import Flask, render_template, request
import base64
import os
import shutil
import threading
import uuid
import time

from src.Image_Classifier.components.data_ingestion import ImageHandler

app = Flask(__name__)

image_handler = ImageHandler()

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        image_data = request.form['image_data']
        folder_name = request.form['folder_name']
        remove_folder_name = request.form.get('remove_folder_name', None)
        saved_images = "path_to_your_saved_images_directory"  # Change this to the actual path
        images = image_handler.process_request(image_data, folder_name, remove_folder_name, saved_images)
        return render_template('index.html', images=images)
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)


In [4]:


# Define directories
saved_images = research\saved_images'
train_dir = r'C:\Users\atulk\PycharmProjects\pythonProject2\D_L\Real_life_project\image_classification\research\train'
test_dir = r'C:\Users\atulk\PycharmProjects\pythonProject2\D_L\Real_life_project\image_classification\research\test'

# Initialize ImageHandler
image_handler = ImageHandler(saved_images, train_dir, test_dir)

# Split and move images
image_handler.split_train_test()


In [3]:
pip install tensorflow


  Using cached tensorflow-2.13.1-cp38-cp38-win_amd64.whl.metadata (2.6 kB)
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorflow-2.13.0-cp38-cp38-win_amd64.whl.metadata (2.6 kB)
  Using cached tensorflow_intel-2.13.0-cp38-cp38-win_amd64.whl.metadata (4.1 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.7-py2.py3-none-any.whl.metadata (849 bytes)
  Using cached gast-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.10.0-cp38-cp38-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached numpy-1.24.3-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 

In [21]:
from tensorflow import keras
from keras import applications, layers
import tensorflow as tf
from keras import applications
from pathlib import Path


In [24]:
from tensorflow.keras import applications

import importlib
model_name = 'VGG16'
model = getattr(applications, model_name)

import importlib



In [29]:
from tensorflow.keras import applications

# Import the desired model
model_name = 'VGG16'
model = getattr(applications, model_name)

# Instantiate the model
model = model(input_shape=[224, 224, 3],
              weights='imagenet',
              include_top=False)

# Save the model
model.save("path_to_save_model.h5")


c:\Users\amank\PycharmProjects\pythonProject2\D_L\Real_life_project\image_classification\env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
applications.l.VGG16

AttributeError: module 'keras.applications' has no attribute 'l'